<a href="https://colab.research.google.com/github/laquita44/big-data-challenge/blob/main/video_dvd_ipynb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# For connection to Postgres run the following code 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-09 02:27:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-04-09 02:27:23 (6.01 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://shoe-inventory-1.s3.us-west-2.amazonaws.com/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# Show Dataframe
video_dvd_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
video_dvd_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...|2015-01-31 00:08:00|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed| 

In [7]:
# Count the number of records (rows) in the dataset
video_dvd_df.count()



5069140

In [13]:
# Drop Null values
video_dvd_df =video_dvd_df.dropna()
video_dvd_df.count()

5068421

In [12]:
# Dropping all duplicates
video_dvd_df= video_dvd_df.dropDuplicates()
print(video_dvd_df.count())

5068421


In [14]:
# check data type for schema
video_dvd_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [15]:
# Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name
from pyspark.sql.types import TimestampType, IntegerType

video_dvd_df = video_dvd_df.withColumn("customer_id", video_dvd_df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", video_dvd_df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", video_dvd_df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", video_dvd_df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", video_dvd_df["helpful_votes"].cast(IntegerType()))\
       .withColumn("total_votes", video_dvd_df["total_votes"].cast(IntegerType()))\

In [16]:
# Create DataFrame to match Schema table "review_id_table"
review_id_table = video_dvd_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1009B8D7RFEI0|   14024341|B007VYEF04|     846810490|2014-01-09 00:07:00|
|R100DVG1NXE56E|   11909676|3898489698|      94759122|2013-01-02 00:03:00|
|R100ILTQB05R4L|   49939297|B008GY8R98|     924905461|2013-01-07 00:01:00|
|R100IT9FL8SC52|   21781238|B0021L8UYY|     155537186|2010-01-26 00:06:00|
|R100JPWF4UCZ8C|   29863328|B0011NVC4S|     465719056|2012-01-26 00:11:00|
|R100P72WHOOF9B|   47809022|B0001DCYN2|      64288116|2014-01-15 00:08:00|
|R100SVJ9ATFTOE|   51130152|B00008AOR6|     629946525|2014-01-07 00:10:00|
|R100YM4OGSIP8I|   12240791|B00SGM38S0|     993804039|2015-01-27 00:01:00|
|R101JSFWMNCW9A|   44680673|B000R7G6K4|     104975481|2009-01-24 00:11:00|
|R101OFQ19FDGRC|   43577749|B000OIOPKO|     685867774|2010-01-10 00:02:00|
|R102088FSMI6PM|   230373

In [18]:
# Create DataFrame to match Schema table "products"
products = video_dvd_df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007VYEF04|Ken Burns: The Du...|
|3898489698|Harun Farocki Fil...|
|B008GY8R98|Die Meistersinger...|
|B0021L8UYY|      The Stepfather|
|B0011NVC4S|SS Hell Pack Trip...|
|B0001DCYN2|        Navajo Blues|
|B00008AOR6|The Righteous Bro...|
|B00SGM38S0|J. Brown's Yoga D...|
|B000R7G6K4|Mr. Magoo's Chris...|
|B000OIOPKO|Hopalong Cassidy ...|
|B000GPIPTM|Scrubs - The Comp...|
|B00077BPD2|John Pinette - I ...|
|B0001ZZ4OG|Fleetwood Mac - U...|
|630528072X|             Volcano|
|B005DZ35UE|Thomas & Friends:...|
|B001TUZI20| Notorious [Blu-ray]|
|B000TKRIVK|    The UFO Cover Up|
|B005LAII2Q|           The Watch|
|6305236569|           Overboard|
|B00006FMB1|Basic Ab Workout ...|
+----------+--------------------+
only showing top 20 rows



In [19]:
#  To create customer_counts that was included in the Schema did a groupby
customers = video_dvd_df.groupby("customer_id").agg({
    "customer_id": "count"
    }).withColumnRenamed("count(customer_id)", "customer_count")
customers.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   17972482|             1|
|   44674947|           228|
|    1465471|            58|
|   29339567|             1|
|   18800155|            35|
|   17669225|             2|
|   37074828|             1|
|   40371408|             6|
|   51680053|            66|
|   12051568|             1|
|   49455997|             1|
|   29399387|             6|
|   25591092|             8|
|   50656151|             1|
|   38726434|             1|
|   50991338|            37|
|   10522786|            10|
|   20349924|           117|
|   51667424|           438|
|   52816423|             3|
+-----------+--------------+
only showing top 20 rows



In [20]:
# Create DataFrame to match Schema table "vine_table"
vine_table = video_dvd_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1009B8D7RFEI0|          5|            0|          0|   N|
|R100DVG1NXE56E|          2|            9|          9|   N|
|R100ILTQB05R4L|          5|            5|          9|   N|
|R100IT9FL8SC52|          2|            1|          2|   N|
|R100JPWF4UCZ8C|          5|            0|          1|   N|
|R100P72WHOOF9B|          5|            0|          0|   N|
|R100SVJ9ATFTOE|          4|            0|          0|   N|
|R100YM4OGSIP8I|          5|            0|          1|   N|
|R101JSFWMNCW9A|          5|            1|          1|   N|
|R101OFQ19FDGRC|          5|            0|          0|   N|
|R102088FSMI6PM|          5|            0|          0|   N|
|R1023GIP6L8IWC|          5|            0|          0|   N|
|R1027RJH8MO373|          3|            0|          1|   N|
|R1028S799242Y6|          3|            

In [28]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://myshoepostgresdb.cgqnrqyjvdcw.us-west-2.rds.amazonaws.com:5432/video_dvd_db"
config = {"user":"root",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [22]:
# Write DataFrame to table
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [24]:
# Drop duplicate values for table "product"
products = products.dropDuplicates()

In [25]:
# Write DataFrame to table
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [26]:
# Write DataFrame to table
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [27]:
# Write DataFrame to table
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)